In [ ]:
# 필요한 라이브러리 불러오기
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import classification_report

# 데이터 로드
data = pd.read_csv("../../../../monunmon.csv")  # 데이터셋 경로 입력
data['Label'] = data['Label'].apply(lambda x: 1 if x >= 0 else 0)  # 이진 레이블 변환

# 특성과 레이블 분리
X = data.drop(columns=['Label'])  # 피처 데이터
y = data['Label']  # 레이블 데이터

# Train, Validation, Test 데이터 나누기
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42, stratify=y_temp)

# 데이터 정규화
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)
X_test = scaler.transform(X_test)

# Gradient Boosting 모델 정의 및 학습
model = GradientBoostingClassifier(random_state=42, n_estimators=100, learning_rate=0.1, max_depth=3)
model.fit(X_train, y_train)

# 테스트 데이터 평가
y_test_pred = model.predict(X_test)

# 분류 보고서 출력
print("\nClassification Report on Test Data:")
print(classification_report(y_test, y_test_pred))



Classification Report on Test Data:
              precision    recall  f1-score   support

           0       0.86      0.28      0.42       450
           1       0.90      0.99      0.94      2850

    accuracy                           0.90      3300
   macro avg       0.88      0.64      0.68      3300
weighted avg       0.89      0.90      0.87      3300



In [ ]:
# 필요한 라이브러리 불러오기
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import classification_report
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler

# 데이터 로드
data = pd.read_csv("../../../../monunmon.csv")  # 데이터셋 경로 입력
data['Label'] = data['Label'].apply(lambda x: 1 if x >= 0 else 0)  # 이진 레이블 변환

# 특성과 레이블 분리
X = data.drop(columns=['Label'])  # 피처 데이터
y = data['Label']  # 레이블 데이터

# Train, Validation, Test 데이터 나누기
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42, stratify=y_temp)

# 데이터 정규화
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)
X_test = scaler.transform(X_test)

### 방법 1: 클래스 가중치 적용 ###
print("=== Gradient Boosting with Class Weights ===")
model_weighted = GradientBoostingClassifier(random_state=42, n_estimators=100)
# Gradient Boosting 자체는 클래스 가중치를 직접 지원하지 않으므로 샘플 가중치를 계산
sample_weight = y_train.map({0: 1/y_train.value_counts()[0], 1: 1/y_train.value_counts()[1]})
model_weighted.fit(X_train, y_train, sample_weight=sample_weight)

# 테스트 데이터 평가
y_test_pred_weighted = model_weighted.predict(X_test)
print("\nClassification Report on Test Data (Class Weights):")
print(classification_report(y_test, y_test_pred_weighted))

### 방법 2: SMOTE를 이용한 오버샘플링 ###
print("\n=== Gradient Boosting with SMOTE ===")
smote = SMOTE(random_state=42)
X_train_smote, y_train_smote = smote.fit_resample(X_train, y_train)

print("Resampled Training Set Size (SMOTE):", X_train_smote.shape)

model_smote = GradientBoostingClassifier(random_state=42, n_estimators=100)
model_smote.fit(X_train_smote, y_train_smote)

# 테스트 데이터 평가
y_test_pred_smote = model_smote.predict(X_test)
print("\nClassification Report on Test Data (SMOTE):")
print(classification_report(y_test, y_test_pred_smote))

### 방법 3: 언더샘플링 ###
print("\n=== Gradient Boosting with Undersampling ===")
undersampler = RandomUnderSampler(random_state=42)
X_train_under, y_train_under = undersampler.fit_resample(X_train, y_train)

print("Resampled Training Set Size (Undersampling):", X_train_under.shape)

model_under = GradientBoostingClassifier(random_state=42, n_estimators=100)
model_under.fit(X_train_under, y_train_under)

# 테스트 데이터 평가
y_test_pred_under = model_under.predict(X_test)
print("\nClassification Report on Test Data (Undersampling):")
print(classification_report(y_test, y_test_pred_under))


=== Gradient Boosting with Class Weights ===

Classification Report on Test Data (Class Weights):
              precision    recall  f1-score   support

           0       0.36      0.72      0.48       450
           1       0.95      0.80      0.86      2850

    accuracy                           0.79      3300
   macro avg       0.65      0.76      0.67      3300
weighted avg       0.87      0.79      0.81      3300


=== Gradient Boosting with SMOTE ===
Resampled Training Set Size (SMOTE): (26600, 15)

Classification Report on Test Data (SMOTE):
              precision    recall  f1-score   support

           0       0.36      0.72      0.48       450
           1       0.95      0.80      0.87      2850

    accuracy                           0.79      3300
   macro avg       0.65      0.76      0.67      3300
weighted avg       0.87      0.79      0.81      3300


=== Gradient Boosting with Undersampling ===
Resampled Training Set Size (Undersampling): (4200, 15)

Classificatio